In [181]:
from pennylane import qchem
from pennylane import numpy as np
from sympy import Matrix

# Configuring The Nuclear Coordinates With Internuclear Radius equal to 1.4, Minimal Basis Set STO-3G and Loading Data from Basis Set Exchange

In [182]:
symbols  = ['H', 'H']
coordinates = np.array([[0.0, 0.0, 0], [0.0, 0.0, 1.4]], requires_grad = False)
H2_molecule = qchem.Molecule(symbols, coordinates, basis_name='STO-3G', load_data=True, unit='bohr')

In [183]:
args = H2_molecule.alpha.copy()
args

[tensor([3.42525091, 0.62391373, 0.1688554 ], requires_grad=False),
 tensor([3.42525091, 0.62391373, 0.1688554 ], requires_grad=False)]

# Calculating Overlap Matrix, Kinetic Matrix, Nuclear Attraction Matrix and Core Hamiltonian

In [184]:
S = Matrix(qchem.overlap_matrix(H2_molecule.basis_set)(*args))
T = Matrix(qchem.kinetic_matrix(H2_molecule.basis_set)(*args))
V = Matrix(qchem.attraction_matrix(H2_molecule.basis_set, H2_molecule.nuclear_charges, H2_molecule.coordinates)(*args))
H = Matrix(qchem.core_matrix(H2_molecule.basis_set, H2_molecule.nuclear_charges, H2_molecule.coordinates)(*args))

In [ ]:
print("1 - Overlap Matrix (S):")
display(S)

print("2 - Kinetic-Energy Matrix (T):")
display(T)

print("3 - Nuclear Attraction Matrix (V):")
display(V)

print("4 - Core Hamiltonian Matrix (H = T + V):")
display(H)

1 - Overlap Matrix (S):


Matrix([
[             1.0, 0.65931820585089],
[0.65931820585089,              1.0]])

2 - Kinetic-Energy Matrix (T):


Matrix([
[0.760031879975584, 0.236454658290793],
[0.236454658290793, 0.760031879975584]])

3 - Nuclear Attraction Matrix (V):


Matrix([
[-1.88044089052278, -1.19483462205357],
[-1.19483462205357, -1.88044089052278]])

4 - Core Hamiltonian Matrix (H = T + V):


Matrix([
[ -1.12040901054719, -0.958379963762777],
[-0.958379963762777,  -1.12040901054719]])

# Performing Restricted Hartree-Fock (RHF) with Pennylane in H2 molecule:

$$FC = SCE$$

In [210]:
hf_eigenvalues, C, F, H, rep_tensor = qchem.scf(H2_molecule, n_steps=10, tol=1e-08)(*args)

print("1 - Coefficient Matrix (C):")
display(Matrix(C))

print("2 - Fock Matrix (F)")
display(Matrix(F))

print(hf_eigenvalues)

1 - Coefficient Matrix (C):


Matrix([
[-0.548934040446712, -1.21146407303756],
[-0.548934040446304,  1.21146407303745]])

2 - Fock Matrix (F)


Matrix([
[-0.365537351430558, -0.593885374732989],
[ -0.59388537473299, -0.365537351430479]])

[-0.57820298  0.67026776]


# Calculating the Ground State Energy for RHF, Nuclear Repulsion Constant and Total Energy

Born-Oppenheimer Electronic Hamiltonian:
$$ 
\mathcal{H}_{\mathrm{elec}} 
= - \sum_{i=1}^N \frac{1}{2}\nabla_i^2
  - \sum_{i=1}^N \sum_{A=1}^N \frac{Z_A}{r_{iA}}
  + \sum_{i=1}^N \sum_{j>i} \frac{1}{r_{ij}}.
$$

Hartree-Fock Ground State:
$$\braket{\psi_{HF}|H_{elec}|\psi_{HF}}=E_0$$

Total Energy (electronic + nuclear repulsion constant):
$$E_{total}=E_0 + \sum_{A=1}^M \sum_{B>A}^M \frac{Z_A \, Z_B}{R_{AB}}$$

In [227]:
E_total = qchem.hf_energy(H2_molecule)(*args)
Nuclear_rep_energy = qchem.nuclear_energy(H2_molecule.nuclear_charges, H2_molecule.coordinates)(*args)[0]
E_0 = E_total- Nuclear_rep_energy

print("Hartree-Fock Ground State Energy: ")
print(f"{E_0:0.4f} a.u\n")

print("Nuclear Repulsion Constant: ")
print(f"{Nuclear_rep_energy:0.4f} a.u\n")

print("Total Energy: ")
print(f"{E_total:0.4f} a.u\n")

Hartree-Fock Ground State Energy: 
-1.8310 a.u

Nuclear Repulsion Constant: 
0.7143 a.u

Total Energy: 
-1.1167 a.u

